# Getting DALEC data into shape for Mortimer's glint correction

In [1]:
import os
import sys
lib_path = os.path.abspath(os.path.join(os.path.abspath(''), 'functions/'))
sys.path.append(lib_path)
lib_path = os.path.abspath(os.path.join(os.path.abspath(''), 'rrs_model_3C-master/'))
sys.path.append(lib_path)
# need to append our functions dir to the path! 
import dalecLoad as dl

import pandas as pd
import numpy as np
import statistics
import matplotlib.pyplot as plt
import spectralConv
import seaborn as sns

import datetime

from scipy.signal import savgol_filter
from scipy import interpolate


from rrs_model_3C import rrs_model_3C

import lmfit as lm

In [2]:
spect_wavelengths = dl.load_DALEC_spect_wavelengths('data/calibrated_full/2023_good_files/LOG_0013.dtf')

In [3]:
log = dl.load_DALEC_dir('data/calibrated_full/2023_good_files/',)

loading ... data/calibrated_full/2023_good_files/LOG_0009.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0035.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0021.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0020.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0034.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0008.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0022.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0036.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0037.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0023.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0027.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0033.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0032.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0026.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0030.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0024.dtf
loading 

loading ... data/calibrated_full/2023_good_files/LOG_0091.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0088.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0077.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0076.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0089.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0048.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0075.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0049.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0065.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0071.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0070.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0064.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0099.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0072.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0066.dtf
loading ... data/calibrated_full/2023_good_files/LOG_0067.dtf
loading 

In [4]:
log.select_dtypes(['number']).columns

Index(['Saturation Flag', 'Solar Elev', 'Spectral Magnitude'], dtype='object')

In [27]:
del log # good to run this to clear memory! 

In [5]:
log_re = dl.resampleMultiLog(log.select_dtypes(['number']), freq='5min')

In [6]:
log_Rrs = dl.uniform_grid_spectra_multi(log_re, spect_wavelengths=spect_wavelengths, resample_to_SDs=False,
                                        Rrs_method='standard')


In [7]:
log_Rrs

Lu_median  Lsky_median  Ed_median  Rrs_median
Date                Wavelength                                               
2023-07-08 08:00:00 400.0        0.002473     0.067946   0.421994    0.001352
                    401.0        0.002536     0.070165   0.437170    0.001307
                    402.0        0.002599     0.071505   0.451485    0.001323
                    403.0        0.002661     0.072844   0.465800    0.001333
                    404.0        0.002719     0.074184   0.474145    0.001354
...                                   ...          ...        ...         ...
2023-09-30 12:50:00 996.0        0.000783     0.028223   0.065525   -0.000103
                    997.0        0.000787     0.028311   0.064959   -0.000091
                    998.0        0.000790     0.028400   0.064341   -0.000079
                    999.0        0.000787     0.028535   0.063635   -0.000193
                    1000.0       0.000783     0.028671   0.062929   -0.000310

[3424498 rows x 4 columns]

In [15]:
Lu = log_Rrs.reset_index().pivot(columns='Wavelength',
                                 index='Date',
                                 values='Lu_median').reset_index().drop('Date', axis=1)
Lu.to_csv('glint_corr/Lu.csv')

In [16]:
Lsky = log_Rrs.reset_index().pivot(columns='Wavelength',
                                 index='Date',
                                 values='Lsky_median').reset_index().drop('Date', axis=1)
Lsky.to_csv('glint_corr/Lsky.csv')

In [17]:
Ed = log_Rrs.reset_index().pivot(columns='Wavelength',
                                 index='Date',
                                 values='Ed_median').reset_index().drop('Date', axis=1)
Ed.to_csv('glint_corr/Ed.csv')

In [25]:
meta = log_Rrs.reset_index().pivot(columns='Wavelength',
                                 index='Date',
                                 values='Lu_median').reset_index()[['Date']]
meta.to_csv('glint_corr/meta.csv')

In [26]:
meta

Wavelength,Date
0,2023-07-08 08:00:00
1,2023-07-08 08:05:00
2,2023-07-08 08:10:00
3,2023-07-08 08:15:00
4,2023-07-08 08:20:00
...,...
5693,2023-09-30 12:30:00
5694,2023-09-30 12:35:00
5695,2023-09-30 12:40:00
5696,2023-09-30 12:45:00
